In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import os
import time
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

In [5]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
print(today)

2021-06-25


In [6]:
!mkdir data_flow

mkdir: data_flow: File exists


In [7]:
#Flow

In [8]:
#https://flowmaps.life.bsc.es/flowboard/static/js/mobility.js
province_id_to_name = {'17': 'Girona', '43': 'Tarragona','08': 'Barcelona', '25': 'Lleida'}

In [9]:
#FLOW-Maps end-points

URL_IN_CCAA = "https://flowmaps.life.bsc.es/api/total_incoming_daily_mobility"
URL_OUT_CCAA = "https://flowmaps.life.bsc.es/api/total_outgoing_daily_mobility" 

INCID_CAT = "https://flowmaps.life.bsc.es/api/incidence"
        
URL_RISK_CCAA = "https://flowmaps.life.bsc.es/api/incoming_risk_history/"

In [10]:
#Load data

In [11]:
payload = {"where":"{\"ev\":\"ES.covid_cpro\",\"start_date\":\"2020-01-01\",\"end_date\":\"" + today + "\"}"}
response = requests.get(INCID_CAT, params=payload, verify = 'flowmaps-life-bsc-es-chain.pem')
print(response.url)

#dfCases["date"] = pd.to_datetime(dfCases["date"])
#dfCases.to_csv("data_flow/cases.csv",index=False)

https://flowmaps.life.bsc.es/api/incidence?where=%7B%22ev%22%3A%22ES.covid_cpro%22%2C%22start_date%22%3A%222020-01-01%22%2C%22end_date%22%3A%222021-06-25%22%7D


In [12]:
print(len(response.json()["_items"]))

dfCases = []
for i,row in enumerate(response.json()["_items"]):
    date = row.get("_id")
    
    dfDaily = pd.DataFrame(row.get("data"))

    dfRow = pd.DataFrame(np.array([date, 
     dfDaily.loc[dfDaily["id"]=="08"]["new_cases"].values[0], 
     dfDaily.loc[dfDaily["id"]=="17"]["new_cases"].values[0],
     dfDaily.loc[dfDaily["id"]=="43"]["new_cases"].values[0],
     dfDaily.loc[dfDaily["id"]=="25"]["new_cases"].values[0],
    ]).reshape(1,5), columns=["date","new_cases_BCN","new_cases_GI","new_cases_TAR","new_cases_LLE"])
    dfCases.append(dfRow)

dfCases = pd.concat(dfCases)
print(len(dfCases))

540
540


In [13]:
dfCases["date"] = pd.to_datetime(dfCases["date"])
dfCases.sort_values("date",ascending=True, inplace=True)

dfCases.to_csv("data_flow/cases_CAT_" + today + ".csv",index=False)

In [14]:
#Gest risk data (Sometimes fails, repeat until it works!)

In [17]:
dfInRisk=[]
for j,val in enumerate(province_id_to_name.keys()):
    payload = {"where":"{\"target\":\"" + val + "\",\"source_layer\":\"cnig_provincias\",\"target_layer\":\"cnig_provincias\",\"ev\":\"ES.covid_cpro\",\"total\":true}"}
    response = requests.get(URL_RISK_CCAA, params=payload, verify = 'flowmaps-life-bsc-es-chain.pem')
    print(response.url)
    df = pd.DataFrame(response.json()["_items"])
    print("CCAA:", val,df.shape)
    
    if len(dfInRisk) == 0:
        dfInRisk = df
        dfInRisk["inrisk_"+val] = dfInRisk["incoming_risk"]
        dfInRisk.date = pd.to_datetime(dfInRisk.date)
        dfInRisk = dfInRisk[["date","inrisk_"+val]]
    else:
        df["inrisk_"+val] = df["incoming_risk"]
        df.date = pd.to_datetime(df.date)
        df = df[["date","inrisk_"+val]]
        dfInRisk = pd.merge(dfInRisk,df,on="date",suffixes=('',''))#,left_index=False,right_index=False) 

#Save
dfInRisk.to_csv("data_flow/inrisk_CAT_"+ today +".csv",index=False)

https://flowmaps.life.bsc.es/api/incoming_risk_history/?where=%7B%22target%22%3A%2217%22%2C%22source_layer%22%3A%22cnig_provincias%22%2C%22target_layer%22%3A%22cnig_provincias%22%2C%22ev%22%3A%22ES.covid_cpro%22%2C%22total%22%3Atrue%7D


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
#LOAD

In [17]:
dfInRisk = pd.read_csv("data_flow/inrisk_CAT_" + today + ".csv")

for i,col in enumerate(dfInRisk.columns):
    if i==0:
        dfInRisk[col] = pd.to_datetime(dfInRisk[col])
    else:
        dfInRisk[col] = pd.to_numeric(dfInRisk[col])

dfInRisk.sort_values("date",ascending=True, inplace=True)
dfInRisk["risk_BCN"] = dfInRisk["inrisk_08"]
dfInRisk["risk_GI"] = dfInRisk["inrisk_17"]
dfInRisk["risk_TAR"] = dfInRisk["inrisk_43"]
dfInRisk["risk_LLE"] = dfInRisk["inrisk_25"]
dfInRisk = dfInRisk[["date","risk_BCN","risk_GI","risk_LLE","risk_TAR"]]

print(dfInRisk.shape)
print(dfInRisk.columns)
print(dfInRisk["date"].iloc[-5:])

(398, 5)
Index(['date', 'risk_BCN', 'risk_GI', 'risk_LLE', 'risk_TAR'], dtype='object')
393   2021-03-15
394   2021-03-16
395   2021-03-17
396   2021-03-18
397   2021-03-19
Name: date, dtype: datetime64[ns]


In [ ]:
dfInRisk.iloc[:,1:].plot(figsize=(15,4))
plt.show()

In [ ]:
# calculate cases by computing the mean of the last 7 days reported cases
cols = dfInRisk.columns
print(cols)
for k in range(1,len(cols)):
    zn = []
    for i in range(dfInRisk.shape[0]):
        acc = 0
        for j in range(7):
            if i-j>=0:
                acc += dfInRisk.iloc[i-j,k]
        zn.append(acc/7)

    dfInRisk[cols[k] + "_7"] = zn
    
    #plot
    #plt.figure(figsize=(10,4))    
    #ax = plt.subplot(1,2,1)
    #dfInRisk[["mean"]].plot(ax=ax)
    #ax = plt.subplot(1,2,2)
    #dfInRisk[["mean_7"]].plot(ax=ax)    
    #plt.show()
    #break
    
dfInRisk = dfInRisk[["date","risk_BCN_7","risk_GI_7","risk_LLE_7","risk_TAR_7"]]
dfInRisk

In [ ]:
dfInRisk.iloc[:,1:].plot(figsize=(15,4))
plt.show()

In [ ]:
#Incidence

In [ ]:
dfCases = pd.read_csv("data_flow/cases_CAT_" + today + ".csv")
dfCases["date"] = pd.to_datetime(dfCases["date"])
dfCases.sort_values("date",ascending=True, inplace=True)

#Aggregate
dfCases["new_cases"] = np.nansum(dfCases[['new_cases_BCN', 'new_cases_GI', 'new_cases_TAR']],axis=1)
dfCases = dfCases[["date","new_cases","new_cases_BCN","new_cases_GI","new_cases_TAR","new_cases_LLE"]]
dfCases["total_cases"] = np.cumsum(dfCases["new_cases"].values)
print(dfCases.shape,dfCases.columns)

In [ ]:
dfCases.iloc[:,2:-1].plot(figsize=(15,4))
plt.show()

In [ ]:
cols = dfCases.columns
for k in range(1,len(cols)):
    zn = []
    for i in range(dfCases.shape[0]):
        acc = 0
        for j in range(7):
            if i-j>=0:
                acc += dfCases.iloc[i-j,k]
        zn.append(acc/7)

    dfCases[cols[k] + "_7"] = zn

dfCases

In [ ]:
dfCases[["new_cases_BCN_7","new_cases_GI_7","new_cases_TAR_7","new_cases_LLE_7"]].plot(figsize=(15,4))
plt.show()

In [ ]:
#Merge cases
dfAll = pd.merge(dfCases[["date","new_cases","total_cases","new_cases_7",
                         "new_cases_GI_7","new_cases_LLE_7","new_cases_BCN_7","new_cases_TAR_7"]],
                 dfInRisk[["date","risk_BCN_7","risk_GI_7","risk_LLE_7","risk_TAR_7"]], 
                 how="left",on="date")
dfAll["date"] = pd.to_datetime(dfAll["date"])

#Na inputation
#dfAll.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True, order=2)


print(dfAll.shape)
print(dfAll.isnull().sum())
dfAll.tail()

In [ ]:
#Remove days without mobility index 
dfAll = dfAll.iloc[46:]

#Remove last day 
dfAll = dfAll.iloc[:-3]

print(dfAll.shape)
print(dfAll.isnull().sum())
dfAll.tail()

In [ ]:
#Na inputation
dfAll.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True, order=2)
print(dfAll.isnull().sum())
dfAll.tail(10)

In [ ]:
#Convert data to format (rows,timepoints,features)
cols = [
         'new_cases',
         'new_cases_7',
         'new_cases_GI_7',
         'new_cases_TAR_7',
         'new_cases_BCN_7',
         'new_cases_LLE_7',    
         'risk_BCN_7',
         'risk_GI_7',
         'risk_LLE_7',
         'risk_TAR_7'
]

In [ ]:
#Forecasting future!!

In [ ]:
res_test = series_to_supervised(dfAll.loc[:,cols].values,21,21,dropnan=False)
print(cols)
res_test.shape

In [ ]:
#Find rows for future pred
ftest_X = res_test.iloc[-1,:210].values.reshape(-1,21,10)
ftest_y = res_test.iloc[-1,[210+(10*i) for i in range(21)]].values.reshape(-1,21)
print(ftest_X.shape, ftest_y.shape)
print(ftest_y[0,:])

In [ ]:
fname = "best_model_flow_provs_CAT_21ahead_1_04022021"

In [ ]:
from pickle import load

# normalize features
for i in range(ftest_X.shape[-1]):

    # normalize features
    scalers_X = load(open("models/"+ fname + "_scaler_tr_X_" + str(i)+'.pkl', 'rb'))

    ftest_X[:,:,i] = scalers_X.transform(ftest_X[:,:,i])

print(ftest_X.shape)
print(np.nanmax(ftest_X),np.nanmin(ftest_X),np.nanmean(ftest_X),np.nanstd(ftest_X))

In [ ]:
#Forecasting last X days for all models

In [ ]:
all_preds = []
for i in range(10):
    print("\nLoading model:", i)
    model = tf.keras.models.load_model("models/"+ fname + "_" + str(i) +'.h5')
    preds = model.predict(ftest_X)
    scaler_y = load(open("models/"+ fname + "_scaler_tr_Y.pkl", 'rb'))

    preds_inv = scaler_y.inverse_transform(preds)
    print(len(preds_inv), np.max(preds_inv), np.min(preds_inv))
    print(preds_inv.shape)
    all_preds.append(preds_inv)

In [ ]:
all_preds_aux = np.stack((all_preds))
print(all_preds_aux.shape)

all_preds_aux[:,-1,:].mean(axis=0)
all_preds_aux[:,-1,:].std(axis=0)

In [ ]:
all_res = all_preds_aux[:,:all_preds_aux.shape[1]]
for i in range(all_preds_aux.shape[1]-1,19,-1):
    all_res = np.hstack((all_preds_aux[:,i,-1].reshape(10,1),all_res))
    #print(all_res.shape)

all_res = np.array(all_res)
print(all_res.shape)

In [ ]:
dfPreds = pd.DataFrame(np.hstack((all_res)).astype(int).reshape(10,21),
                       columns=["d"+str(i) for i in range(21)])
dfPreds.astype(int).to_csv("preds_infected_21days" + today + ".csv",index=False)
dfPreds.describe().T

In [ ]:
import datetime
import matplotlib.dates as mdates

date_time_str = '2021-04-07' #######Last data collected day

base = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')
date_list = [base + datetime.timedelta(days=x) for x in range(21)]

In [ ]:
dfPredsT = dfPreds.T
dfPredsT.reset_index(drop=True, inplace=True)
dfPredsT["date"] = date_list
dfPredsT.head()

In [ ]:
print(dfPredsT[[i for i in range(10)]].mean(axis=1).mean(axis=0))
print(dfPredsT[[i for i in range(10)]].mean(axis=1).std(axis=0))
print(np.mean(dfPredsT[[i for i in range(10)]].values),
      np.std(dfPredsT[[i for i in range(10)]].values))

In [ ]:
plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1)
ax.plot(dfPredsT["date"], dfPredsT[[i for i in range(10)]])
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.title("Daily predicted infected cases")
plt.xticks(rotation='vertical')
plt.ylim(500,3500)
plt.show()

plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1)
ax.errorbar(dfPredsT["date"], 
            dfPredsT[[i for i in range(10)]].mean(axis=1),
            yerr=dfPredsT[[i for i in range(10)]].std(axis=1))
ax.fill_between(dfPredsT["date"],
                dfPredsT[[i for i in range(10)]].mean(axis=1)-dfPredsT[[i for i in range(10)]].std(axis=1),
                dfPredsT[[i for i in range(10)]].mean(axis=1)+dfPredsT[[i for i in range(10)]].std(axis=1),
                alpha=0.2)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.title("Daily predicted infected cases")
plt.xticks(rotation='vertical')
plt.ylim(500,3500)
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
ax = plt.subplot(1,1,1)
ax.bar(dfPredsT["date"].values, np.mean(dfPredsT[[i for i in range(10)]], axis=1), 
            yerr=np.std(dfPredsT[[i for i in range(10)]], axis=1), label="COVID-19 daily infected cases")
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.title("Daily predicted infected cases")
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1)
ax.plot(dfPredsT["date"], dfPredsT[[i for i in range(10)]].cumsum())
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.title("Daily predicted accumualted infected cases")
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1)
#ax.plot(dfPredsT["date"], dfPredsT[[i for i in range(10)]].cumsum())
ax.errorbar(dfPredsT["date"], 
            dfPredsT[[i for i in range(10)]].cumsum().mean(axis=1),
            yerr=dfPredsT[[i for i in range(10)]].cumsum().std(axis=1))
ax.fill_between(dfPredsT["date"],
                dfPredsT[[i for i in range(10)]].cumsum().mean(axis=1)-dfPredsT[[i for i in range(10)]].cumsum().std(axis=1),
                dfPredsT[[i for i in range(10)]].cumsum().mean(axis=1)+dfPredsT[[i for i in range(10)]].cumsum().std(axis=1),
                alpha=0.2)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.title("Daily predicted accumualted infected cases")
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
from scipy.stats import gamma, poisson

import epyestim
import epyestim.covid19 as covid19

In [ ]:
ch_cases = pd.read_csv(
    "data_flow/cases_CAT_" + today + ".csv",
    parse_dates=['date']
).set_index('date')['new_cases_BCN']


In [ ]:
si_distrb = covid19.generate_standard_si_distribution()
delay_distrb = covid19.generate_standard_infection_to_reporting_distribution()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,3))

axs[0].bar(range(len(si_distrb)), si_distrb, width=1)
axs[1].bar(range(len(delay_distrb)), delay_distrb, width=1)

axs[0].set_title('Default serial interval distribution')
axs[1].set_title('Default infection-to-reporting delay distribution')
plt.show()

In [ ]:
my_continuous_distrb = gamma(a=5, scale=2)
my_discrete_distrb = epyestim.discrete_distrb(my_continuous_distrb)

plt.bar(range(len(my_discrete_distrb)), my_discrete_distrb, width=1)
plt.show()

In [ ]:
ch_time_varying_r = covid19.r_covid(ch_cases)

ch_time_varying_r.tail()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))

ch_time_varying_r.loc[:,'Q0.5'].plot(ax=ax, color='red')
ax.fill_between(ch_time_varying_r.index, 
                    ch_time_varying_r['Q0.025'], 
                    ch_time_varying_r['Q0.975'], 
                    color='red', alpha=0.2)
ax.set_xlabel('date')
ax.set_ylabel('R(t) with 95%-CI')
ax.set_ylim([0,3])
ax.axhline(y=1)
ax.set_title('Estimate of time-varying effective reproduction number for Barcelona')
plt.show()

In [ ]:
ch_time_varying_r_ = covid19.r_covid(ch_cases, smoothing_window=21, r_window_size=7)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))

ch_time_varying_r_.loc[:,'Q0.5'].plot(ax=ax, color='orange')
ax.fill_between(ch_time_varying_r_.index, 
                    ch_time_varying_r_['Q0.025'], 
                    ch_time_varying_r_['Q0.975'], 
                    color='orange', alpha=0.2)
ax.set_xlabel('date')
ax.set_ylabel('R(t) with 95%-CI')
ax.set_ylim([0,3])
ax.axhline(y=1)
ax.set_title('Estimate of time-varying effective reproduction number for Barcelona')
plt.show()
ch_time_varying_r.tail()

In [ ]:
cases_pred = dfPredsT[[i for i in range(10)]].mean(axis=1)
d = {'date': dfPredsT["date"], 'cases': cases_pred}
#print(d)
#print(cases_pred)
#print(dfPredsT)
#tmp = [dfPredsT["date"].values, cases_pred]
ch_cases_pred = pd.DataFrame(data = d).set_index('date')
print(type(ch_cases_pred))
#ch_time_varying_r = covid19.r_covid(ch_cases_pred)
#ch_time_varying_r.tail()
#ch_cases_pred = pd.read_csv("predict_tmp.csv",
#                         parse_dates=['date']).set_index('date')['cases']

In [ ]:
ch_time_varying = covid19.r_covid(ch_cases_pred.squeeze())

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))

ch_time_varying.loc[:,'Q0.5'].plot(ax=ax, color='orange')
ax.fill_between(ch_time_varying.index, 
                    ch_time_varying['Q0.025'], 
                    ch_time_varying['Q0.975'], 
                    color='orange', alpha=0.2)
ax.set_xlabel('date')
ax.set_ylabel('R(t) with 95%-CI')
ax.set_ylim([0.90,1.10])
ax.axhline(y=1)
ax.set_title('Estimate of time-varying effective reproduction number for Barcelona')
plt.show()

In [ ]:
print(ch_time_varying)

In [ ]:
print((1.15-1.013)/1.15)